# SQL query from table names - Continued

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [10]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    },
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [{'role': 'system','content': """
CREATE TABLE Users (
    user_id INT PRIMARY KEY,
    username VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    description TEXT,
    price DECIMAL(10, 2) NOT NULL,
    stock INT DEFAULT 0
);

CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    user_id INT,
    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    status VARCHAR(20) DEFAULT 'Pending',
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
);

CREATE TABLE OrderItems (
    order_item_id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT NOT NULL,
    price DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);

CREATE TABLE Reviews (
    review_id INT PRIMARY KEY,
    user_id INT,
    product_id INT,
    rating INT CHECK (rating BETWEEN 1 AND 5),
    comment TEXT,
    review_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);
"""
    }
]

In [4]:
#FEW SHOT SAMPLES
context.append({'role': 'system','content': """
-- Maintain the SQL order simple and efficient as you can, using valid SQLite.
-- Answer the following questions for the tables provided above.

-- 1. List all users who have placed at least one order.
SELECT DISTINCT u.user_id, u.username, u.email
FROM Users u
JOIN Orders o ON u.user_id = o.user_id;

-- 2. Get the total number of products in stock.
SELECT SUM(stock) AS total_stock FROM Products;

-- 3. Find all orders and their total item count.
SELECT o.order_id, COUNT(oi.order_item_id) AS total_items
FROM Orders o
JOIN OrderItems oi ON o.order_id = oi.order_id
GROUP BY o.order_id;

-- 4. Show the average rating of each product that has received reviews.
SELECT p.product_id, p.name, AVG(r.rating) AS avg_rating
FROM Products p
JOIN Reviews r ON p.product_id = r.product_id
GROUP BY p.product_id;

-- 5. List the top 3 most recently created users.
SELECT * FROM Users
ORDER BY created_at DESC
LIMIT 3;
"""
})


In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [6]:
#new
context_user = context.copy()
print("This is your SQL:\n")
print(return_CCRMSQL("""
This is your SQL, after that an SQL that can do what the user request.
SELECT * 
FROM Users 
WHERE created_at >= datetime('now', '-7 days') 
ORDER BY created_at DESC;
""", context_user))

This is your SQL:

```sql
-- Retrieve users who have registered in the last 7 days and order them by the registration date in descending order.
SELECT *
FROM Users
WHERE created_at >= datetime('now', '-7 days')
ORDER BY created_at DESC;
```


In [7]:
#old
old_context_user = old_context.copy()
#print("This is your SQL:\n")
print(return_CCRMSQL("""
SELECT * 
FROM Users 
WHERE created_at < datetime('now', '-1 year') 
ORDER BY created_at ASC;
""", old_context_user))

This is your SQL:
```sql
SELECT * 
FROM Users 
WHERE created_at < datetime('now', '-1 year') 
ORDER BY created_at ASC;
```
Explanation: This SQL query selects all records from the "Users" table where the "created_at" date is older than one year ago, ordering the results in ascending order based on the "created_at" column. 




In [8]:
#new
context_user = context.copy()
print("This is your SQL:\n")
print(return_CCRMSQL("""
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN employees e ON st.ID_Usr = e.ID_Usr
JOIN salary sa ON e.ID_Usr = sa.ID_Usr
GROUP BY st.Institution
ORDER BY avg_salary DESC
LIMIT 1;
""", context_user))

This is your SQL:

The query you provided seems to be referencing tables and columns that are not defined in the initial schema provided. To assist you further, please provide the schema for the tables `studies`, `employees`, and `salary` along with sample data. This will help in understanding the structure of the tables and the relationships between them. Once you provide that information, I can help you formulate the query to achieve the desired result.


In [9]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql

SELECT * 
FROM employees;
```
This SQL query selects all data from the "employees" table.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

## Version 1: Manual SQL Writing

* Purpose: Execute hand-written SQL queries against sample data.
* How it works:
    * Uses sqlite3 to create in-memory tables (employees, salary, etc.).
    * Executes static SQL queries using pandas.read_sql_query().
* Use Case: For validation/testing or training purposes where the SQL is predefined.
* Strengths: Fast and no dependency on OpenAI.
* Limitation: No AI-powered query generation.

In [11]:
import sqlite3
import pandas as pd

In [12]:
# Create in-memory SQLite DB
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create tables
cursor.execute("""
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,
    name TEXT
);
""")

cursor.execute("""
CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT,
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);
""")

# Insert sample data
cursor.executemany("INSERT INTO employees VALUES (?, ?)", [
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie')
])

cursor.executemany("INSERT INTO salary VALUES (?, ?, ?)", [
    (1, '2023-01-01', 60000),
    (2, '2023-01-01', 48000),
    (3, '2023-01-01', 52000)
])

conn.commit()


In [13]:
query = """
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary > 50000;
"""

df = pd.read_sql_query(query, conn)
df

,name,salary
0,Alice,60000.0
1,Charlie,52000.0


## Version 2: Static Prompt

* Purpose: Generate SQL using a fixed natural language prompt and predefined schema context.
* How it works:
    * Uses the OpenAI SDK (openai>=1.0.0) with OpenAI(api_key=...).
    * Context is defined with schema and few-shot examples.
    * Static user prompt like “Which institution has the highest average salary?” is passed into return_CCRMSQL(prompt, context).
* Use Case: Template-based query generation for fixed questions.
* Strengths: Accurate SQL generation with consistent prompts.
* Limitation: Requires code edit to change input prompt.

In [18]:
import openai

In [19]:
context = [{
    'role': 'system',
    'content': """You are a helpful assistant that writes SQL queries. Use this schema:
CREATE TABLE Users (
    user_id INT PRIMARY KEY,
    username VARCHAR(50),
    email VARCHAR(100),
    created_at TIMESTAMP
);
CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    user_id INT,
    order_date TIMESTAMP,
    status VARCHAR(20),
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
);"""
}]

In [20]:
context.append({
    'role': 'user',
    'content': "Show all users who placed more than 2 orders."
})

In [23]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [25]:
context_user = context.copy()
print("This is your SQL:\n")
print(return_CCRMSQL("""
This is your SQL, after that an SQL that can do what the user request.
SELECT * 
FROM Users 
WHERE created_at >= datetime('now', '-7 days') 
ORDER BY created_at DESC;
""", context_user))

This is your SQL:

To show all users who placed more than 2 orders, you can use the following SQL query:

```sql
SELECT u.user_id, u.username, u.email, COUNT(o.order_id) AS total_orders
FROM Users u
JOIN Orders o ON u.user_id = o.user_id
GROUP BY u.user_id
HAVING COUNT(o.order_id) > 2;
``` 

This query will retrieve the user_id, username, email, and the total number of orders placed by each user. It will then filter out users who have placed more than 2 orders.


## Dynamic Prompt Interface

* Purpose: Provide an interactive interface to enter natural language prompts dynamically.
* How it works:
    * Uses ipywidgets (Textarea, Button, Output) for prompt input.
    * User types in a question (e.g., "List top 3 products by average rating").
    * Button triggers return_CCRMSQL() which sends prompt to OpenAI and displays generated SQL.
* Use Case: Real-time experimentation or end-user-friendly notebooks.
* Strengths: No code edits needed; intuitive UX.
* Limitation: Requires running Jupyter environment with widget support.

In [26]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [27]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Use the same context defined earlier in the notebook
context_user = context.copy()

# Input widget
input_box = widgets.Textarea(
    value='',
    placeholder='Type your question here...',
    description='Prompt:',
    layout=widgets.Layout(width='100%', height='100px')
)

# Button widget
button = widgets.Button(description='Generate SQL')

# Output area
output = widgets.Output()

# Callback function when button is clicked
def on_button_click(b):
    output.clear_output()
    user_prompt = input_box.value.strip()
    if user_prompt:
        with output:
            print("⏳ Generating SQL for your prompt...")
            try:
                response = return_CCRMSQL(user_prompt, context_user)
                display(Markdown(f"### ✅ Generated SQL:\n```\n{response}\n```"))
            except Exception as e:
                print(f"❌ Error: {e}")
    else:
        with output:
            print("⚠️ Please enter a prompt.")

button.on_click(on_button_click)

# Display UI
display(input_box, button, output)


Textarea(value='', description='Prompt:', layout=Layout(height='100px', width='100%'), placeholder='Type your …

Button(description='Generate SQL', style=ButtonStyle())

Output()

## Observations
* The approach demonstrates the power of combining schema grounding, few-shot examples, and LLMs to translate real-world questions into valid SQL.
* Version 3 offers the most user-friendly experience, especially for non-technical users.
* Context accuracy and model temperature have strong impact on SQL quality.
* Modular design using return_CCRMSQL makes the system extensible.

* Conclusion: Adopt Version 3 as the primary interaction interface, and optionally retain Version 1 for validation/testing purposes. Continue iterating on context quality and example diversity for more robust SQL generation.